In [1]:
from __future__ import print_function

import argparse
import os
import shutil
import time
import random

import torch
import torch.nn as nn

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets


from utils import Bar, Logger, AverageMeter, accuracy, mkdir_p, savefig
import numpy as np

In [2]:
# Declaring torch variables
use_cuda = torch.cuda.is_available()

manualSeed = random.randint(1, 10000)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if use_cuda:
    torch.cuda.manual_seed_all(manualSeed)

best_acc = 0  # best test accuracy

In [3]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def alexnet(**kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    """
    model = AlexNet(**kwargs)
    return model

In [4]:
class InferenceAttack_HZ(nn.Module):
    def __init__(self,num_classes):
        self.num_classes=num_classes
        super(InferenceAttack_HZ, self).__init__()
        self.features=nn.Sequential(
            nn.Linear(100,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,64),
            nn.ReLU(),
            )
        self.labels=nn.Sequential(
           nn.Linear(num_classes,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            )
        self.combine=nn.Sequential(
            nn.Linear(64*2,256),
            
            nn.ReLU(),
            nn.Linear(256,128),
            
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,1),
            )
        for key in self.state_dict():
            print (key)
            if key.split('.')[-1] == 'weight':    
                nn.init.normal(self.state_dict()[key], std=0.01)
                print (key)
                
            elif key.split('.')[-1] == 'bias':
                self.state_dict()[key][...] = 0
        self.output= nn.Sigmoid()
    def forward(self,x,l):
        
        out_x = self.features(x)
        out_l = self.labels(l)
        
        is_member =self.combine( torch.cat((out_x  ,out_l),1))
        
        return self.output(is_member)

In [5]:
dataset='cifar100'
checkpoint_path='/home/ymohit/Robust-MIA/logs/milad/checkpoint/checkpoints_100cifar_alexnet_white'
train_batch=100
test_batch=100
lr=0.05
epochs=500
state={}
state['lr']=lr

In [6]:

def train_privatly(trainloader, model,inference_model, criterion, optimizer, epoch, use_cuda,num_batchs=10000,alpha=0.9):
    # switch to train mode
    model.train()
    inference_model.eval()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()
    

    first_id = -1
    for batch_idx, (inputs, targets) in (trainloader):
        # measure data loading time
        data_time.update(time.time() - end)
        if first_id == -1:
            first_id = batch_idx
        
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((outputs.size(0),100))-1)).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)

        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
        
        inference_output = inference_model ( outputs,infer_input_one_hot)
        #print (inference_output.mean())
        

        loss = criterion(outputs, targets) + (alpha)*(((inference_output-1.0).pow(2).mean()))

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
#         losses.update(loss.data[0], inputs.size(0))
#         top1.update(prec1[0], inputs.size(0))
#         top5.update(prec5[0], inputs.size(0))

        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))


        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if batch_idx%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=batch_idx + 1,
                    size=500,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
        if batch_idx-first_id >= num_batchs:
            break

    return (losses.avg, top1.avg)

In [7]:
def train(trainloader, model, criterion, optimizer, epoch, use_cuda):
    # switch to train mode
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    bar = Bar('Processing', max=len(trainloader))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)

        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
#         losses.update(loss.data[0], inputs.size(0))
#         top1.update(prec1[0], inputs.size(0))
#         top5.update(prec5[0], inputs.size(0))

        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if batch_idx%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=batch_idx + 1,
                    size=len(trainloader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))

    return (losses.avg, top1.avg)

In [8]:
def test(testloader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
#         losses.update(loss.data[0], inputs.size(0))
#         top1.update(prec1[0], inputs.size(0))
#         top5.update(prec5[0], inputs.size(0))

        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if batch_idx % 100==0:
            
            print ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                        batch=batch_idx + 1,
                        size=len(testloader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        loss=losses.avg,
                        top1=top1.avg,
                        top5=top5.avg,
                        ))

    return (losses.avg, top1.avg)

In [ ]:
def privacy_train(trainloader, model,inference_model, criterion, optimizer, epoch, use_cuda,num_batchs=1000):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    mtop1_a = AverageMeter()
    mtop5_a = AverageMeter()
    
    inference_model.train()
    model.eval()
    # switch to evaluate mode

    end = time.time()
    first_id = -1
    for batch_idx,((tr_input, tr_target) ,(te_input, te_target)) in trainloader:
        # measure data loading time
        if first_id == -1:
            first_id = batch_idx
        
        data_time.update(time.time() - end)
        tr_input = tr_input.cuda()
        te_input = te_input.cuda()
        tr_target = tr_target.cuda()
        te_target = te_target.cuda()
        
        
        v_tr_input = torch.autograd.Variable(tr_input)
        v_te_input = torch.autograd.Variable(te_input)
        v_tr_target = torch.autograd.Variable(tr_target)
        v_te_target = torch.autograd.Variable(te_target)
        
        # compute output
        model_input =torch.cat((v_tr_input,v_te_input))
        
        pred_outputs = model(model_input)
        
        infer_input= torch.cat((v_tr_target,v_te_target))
        
        mtop1, mtop5 =accuracy(pred_outputs.data, infer_input.data, topk=(1, 5))
        
#         mtop1_a.update(mtop1[0], model_input.size(0))
#         mtop5_a.update(mtop5[0], model_input.size(0))

                
        mtop1_a.update(mtop1.item(), model_input.size(0))
        mtop5_a.update(mtop5.item(), model_input.size(0))
        
        one_hot_tr = torch.from_numpy((np.zeros((infer_input.size(0),100))-1)).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, infer_input.type(torch.cuda.LongTensor).view([-1,1]).data,1)

        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
        

        attack_model_input = pred_outputs#torch.cat((pred_outputs,infer_input_one_hot),1)
        member_output = inference_model(attack_model_input,infer_input_one_hot)
        
        
        
        is_member_labels = torch.from_numpy(np.reshape(np.concatenate((np.zeros(v_tr_input.size(0)),np.ones(v_te_input.size(0)))),[-1,1])).cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels).type(torch.cuda.FloatTensor)

        loss = criterion(member_output, v_is_member_labels)

        # measure accuracy and record loss
        prec1=np.mean((member_output.data.cpu().numpy() >0.5)==v_is_member_labels.data.cpu().numpy())
        #losses.update(loss.data[0], model_input.size(0))
        
        losses.update(loss.item(), model_input.size(0))
        
        top1.update(prec1, model_input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx-first_id > num_batchs:
            break

        # plot progress
        if batch_idx%10==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx ,
                    size=500,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))

    return (losses.avg, top1.avg)

In [ ]:
def privacy_test(trainloader, model,inference_model, criterion, optimizer, epoch, use_cuda,num_batchs=1000):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    mtop1_a = AverageMeter()
    mtop5_a = AverageMeter()
    
    inference_model.eval()
    model.eval()
    # switch to evaluate mode

    end = time.time()
    for batch_idx,((tr_input, tr_target) ,(te_input, te_target)) in trainloader:
        # measure data loading time
        if first_id == -1:
            first_id = batch_idx

        data_time.update(time.time() - end)
        tr_input = tr_input.cuda()
        te_input = te_input.cuda()
        tr_target = tr_target.cuda()
        te_target = te_target.cuda()
        
        
        v_tr_input = torch.autograd.Variable(tr_input)
        v_te_input = torch.autograd.Variable(te_input)
        v_tr_target = torch.autograd.Variable(tr_target)
        v_te_target = torch.autograd.Variable(te_target)
        
        # compute output
        model_input =torch.cat((v_tr_input,v_te_input))
        
        pred_outputs = model(model_input)
        
        infer_input= torch.cat((v_tr_target,v_te_target))
        
        mtop1, mtop5 =accuracy(pred_outputs.data, infer_input.data, topk=(1, 5))
        
#         mtop1_a.update(mtop1[0], model_input.size(0))
#         mtop5_a.update(mtop5[0], model_input.size(0))

        mtop1_a.update(mtop1.item(), model_input.size(0))
        mtop5_a.update(mtop5.item(), model_input.size(0))
        
        
        one_hot_tr = torch.from_numpy((np.zeros((infer_input.size(0),100))-1)).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, infer_input.type(torch.cuda.LongTensor).view([-1,1]).data,1)

        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
        

        attack_model_input = pred_outputs#torch.cat((pred_outputs,infer_input_one_hot),1)
        member_output = inference_model(attack_model_input,infer_input_one_hot)
        
        
        
        is_member_labels = torch.from_numpy(np.reshape(np.concatenate((np.zeros(v_tr_input.size(0)),np.ones(v_te_input.size(0)))),[-1,1])).cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels).type(torch.cuda.FloatTensor)

        loss = criterion(member_output, v_is_member_labels)

        # measure accuracy and record loss
        prec1=np.mean((member_output.data.cpu().numpy() >0.5)==v_is_member_labels.data.cpu().numpy())
        #losses.update(loss.data[0], model_input.size(0))
        
        losses.update(loss.item(), model_input.size(0))
        
        top1.update(prec1, model_input.size(0))

        # compute gradient and do SGD step
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx-first_id >= num_batchs:
            break

        # plot progress
#         if batch_idx%10==0:
#             print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
#                     batch=batch_idx ,
#                     size=len(trainloader),
#                     data=data_time.avg,
#                     bt=batch_time.avg,
#                     loss=losses.avg,
#                     top1=top1.avg,
#                     ))

    return (losses.avg, top1.avg)


In [ ]:

def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in [20,40]:
        state['lr'] *= 0.1 
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']


def save_checkpoint_adversary(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_adversary_best.pth.tar'))

In [ ]:
global best_acc
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

if not os.path.isdir(checkpoint_path):
    mkdir_p(checkpoint_path)

In [ ]:
print('==> Preparing dataset %s' % dataset)
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

transform_train = transforms.Compose([
        #transforms.RandomCrop(32, padding=4),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    
    
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

# prepare test data parts
transform_test = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean, std)])
    

if dataset == 'cifar10':
    dataloader = datasets.CIFAR10
    num_classes = 10
else:
    dataloader = datasets.CIFAR100
    num_classes = 100



# Model
print("==> creating model ")


model = AlexNet(num_classes)


# In[51]:


model = model.cuda()

==> Preparing dataset cifar100
==> creating model 


In [ ]:
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

criterion = nn.CrossEntropyLoss()
criterion_attack = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    Total params: 2.50M


In [ ]:
# Resume
title = 'cifar-100'


# In[38]:


inferenece_model = InferenceAttack_HZ(100).cuda()
private_train_criterion = nn.MSELoss()


# In[39]:


optimizer_mem = optim.Adam(inferenece_model.parameters(), lr=0.00001)

features.0.weight
features.0.weight
features.0.bias
features.2.weight
features.2.weight
features.2.bias
features.4.weight
features.4.weight
features.4.bias
labels.0.weight
labels.0.weight
labels.0.bias
labels.2.weight
labels.2.weight
labels.2.bias
combine.0.weight
combine.0.weight
combine.0.bias
combine.2.weight
combine.2.weight
combine.2.bias
combine.4.weight
combine.4.weight
combine.4.bias
combine.6.weight
combine.6.weight
combine.6.bias


/home/ymohit/anaconda3/envs/rmia/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.


In [ ]:
data_100_dir = '/home/ymohit/Robust-MIA/logs/milad/data100'


batch_privacy=100
trainset = dataloader(root=data_100_dir, train=True, download=True, transform=transform_train)
trainloader = data.DataLoader(trainset, batch_size=batch_privacy, shuffle=True, num_workers=1)

trainset_private = dataloader(root=data_100_dir, train=True, download=True, transform=transform_test)
trainloader_private = data.DataLoader(trainset, batch_size=batch_privacy, shuffle=True, num_workers=1)

testset = dataloader(root=data_100_dir, train=False, download=False, transform=transform_test)
testloader = data.DataLoader(testset, batch_size=batch_privacy, shuffle=True, num_workers=1)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
batch_privacy=100
trainset = dataloader(root=data_100_dir, train=True, download=True, transform=transform_test)
testset = dataloader(root=data_100_dir, train=False, download=False, transform=transform_test)

r = np.arange(50000)
np.random.shuffle(r)

private_trainset_intrain = []
private_trainset_intest = []

private_testset_intrain =[] 
private_testset_intest =[] 


for i in range(25000):
    private_trainset_intrain.append(trainset[r[i]])

for i in range(25000,50000):
    private_testset_intrain.append(trainset[r[i]])

    
r = np.arange(10000)
np.random.shuffle(r)
  
for i in range(5000):
    private_trainset_intest.append(testset[r[i]])


for i in range(5000,10000):
    private_testset_intest.append(testset[r[i]])


Files already downloaded and verified


In [ ]:
private_trainloader_intrain = data.DataLoader(private_trainset_intrain, batch_size=batch_privacy, shuffle=True, num_workers=1)
private_trainloader_intest = data.DataLoader(private_trainset_intest, batch_size=batch_privacy, shuffle=True, num_workers=1)


private_testloader_intrain = data.DataLoader(private_testset_intrain, batch_size=batch_privacy, shuffle=True, num_workers=1)
private_testloader_intest = data.DataLoader(private_testset_intest, batch_size=batch_privacy, shuffle=True, num_workers=1)

In [ ]:
is_best=False
best_acc=0.0
start_epoch=0
# Train and val
for epoch in range(start_epoch, 400):
    adjust_learning_rate(optimizer, epoch)

    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))

    train_enum = enumerate(trainloader)
    train_private_enum = enumerate(zip(trainloader_private,testloader))
    for i in range(500//2):
        
        
        if epoch>3:
            privacy_loss, privacy_acc = privacy_train(train_private_enum,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda,1)
            train_loss, train_acc = train_privatly(train_enum, model,inferenece_model, criterion, optimizer, epoch, use_cuda,1,1)
            
            
            if i%10 ==0:
                print ('privacy res',privacy_acc,train_acc)
            if  (i+1)%50 ==0:
                train_private_enum = enumerate(zip(trainloader_private,testloader))
        else:
            train_loss, train_acc = train_privatly(train_enum, model,inferenece_model, criterion, optimizer, epoch, use_cuda,1000,0)
            break
        
        
    test_loss, test_acc = test(testloader, model, criterion, epoch, use_cuda)

    print ('test acc',test_acc)

    # save model
    is_best = test_acc>best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, False, checkpoint=checkpoint_path,filename='epoch%d'%epoch)

print('Best acc:')
print(best_acc)


Epoch: [1 | 500] LR: 0.050000
(1/500) Data: 0.005s | Batch: 0.479s | | Loss: 4.6074 | top1:  1.0000 | top5:  3.0000
(101/500) Data: 0.016s | Batch: 0.031s | | Loss: 4.5981 | top1:  1.1485 | top5:  6.0792
(201/500) Data: 0.015s | Batch: 0.027s | | Loss: 4.4941 | top1:  2.1642 | top5:  9.6219
(301/500) Data: 0.015s | Batch: 0.026s | | Loss: 4.3810 | top1:  3.1063 | top5:  13.2226
(401/500) Data: 0.014s | Batch: 0.026s | | Loss: 4.2870 | top1:  4.0100 | top5:  16.2818


/home/ymohit/anaconda3/envs/rmia/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


(1/100) Data: 0.153s | Batch: 0.161s | Loss: 3.9054 | top1:  6.0000 | top5:  30.0000
test acc 8.34

Epoch: [2 | 500] LR: 0.050000
(1/500) Data: 0.003s | Batch: 0.020s | | Loss: 3.7342 | top1:  14.0000 | top5:  38.0000
(101/500) Data: 0.014s | Batch: 0.025s | | Loss: 3.8011 | top1:  10.4158 | top5:  32.4752
(201/500) Data: 0.013s | Batch: 0.024s | | Loss: 3.7498 | top1:  11.0647 | top5:  34.0647
(301/500) Data: 0.013s | Batch: 0.023s | | Loss: 3.7204 | top1:  11.7375 | top5:  35.0631
(401/500) Data: 0.013s | Batch: 0.023s | | Loss: 3.6875 | top1:  12.3516 | top5:  36.1496
(1/100) Data: 0.154s | Batch: 0.160s | Loss: 3.5808 | top1:  15.0000 | top5:  36.0000
test acc 15.09

Epoch: [3 | 500] LR: 0.050000
(1/500) Data: 0.003s | Batch: 0.025s | | Loss: 3.4113 | top1:  16.0000 | top5:  45.0000
(101/500) Data: 0.012s | Batch: 0.023s | | Loss: 3.4172 | top1:  17.0297 | top5:  44.4356
(201/500) Data: 0.012s | Batch: 0.022s | | Loss: 3.3923 | top1:  17.4677 | top5:  44.9900
(301/500) Data: 0.012s

(40/500) Data: 0.004s | Batch: 0.017s | | Loss: 0.2443 | top1:  0.5975 
(60/500) Data: 0.004s | Batch: 0.016s | | Loss: 0.2470 | top1:  0.5200 
privacy res 0.545 30.0
(70/500) Data: 0.004s | Batch: 0.017s | | Loss: 0.2448 | top1:  0.5650 
(90/500) Data: 0.005s | Batch: 0.021s | | Loss: 0.2480 | top1:  0.5300 
privacy res 0.5783333333333334 24.5
privacy res 0 25.0
(0/500) Data: 0.005s | Batch: 0.028s | | Loss: 0.2456 | top1:  0.5600 
(401/500) Data: 0.001s | Batch: 0.014s | | Loss: 3.0652 | top1:  31.0000 | top5:  64.0000
privacy res 0.5416666666666666 33.5
(10/500) Data: 0.004s | Batch: 0.025s | | Loss: 0.2460 | top1:  0.5675 
(30/500) Data: 0.004s | Batch: 0.021s | | Loss: 0.2470 | top1:  0.5700 
privacy res 0.5433333333333333 26.5
(40/500) Data: 0.004s | Batch: 0.019s | | Loss: 0.2388 | top1:  0.5925 
(60/500) Data: 0.004s | Batch: 0.022s | | Loss: 0.2393 | top1:  0.6100 
privacy res 0.5966666666666667 29.5
(70/500) Data: 0.003s | Batch: 0.017s | | Loss: 0.2436 | top1:  0.5675 
(90/5

(10/500) Data: 0.004s | Batch: 0.019s | | Loss: 0.2421 | top1:  0.5525 
(30/500) Data: 0.004s | Batch: 0.016s | | Loss: 0.2346 | top1:  0.5900 
privacy res 0.5766666666666667 31.0
(40/500) Data: 0.003s | Batch: 0.016s | | Loss: 0.2359 | top1:  0.6025 
(60/500) Data: 0.003s | Batch: 0.017s | | Loss: 0.2488 | top1:  0.5200 
privacy res 0.5533333333333333 27.5
(70/500) Data: 0.003s | Batch: 0.017s | | Loss: 0.2416 | top1:  0.5550 
(90/500) Data: 0.003s | Batch: 0.015s | | Loss: 0.2446 | top1:  0.5550 
privacy res 0.5516666666666666 28.0
privacy res 0 31.0
(1/100) Data: 0.101s | Batch: 0.107s | Loss: 3.3565 | top1:  31.0000 | top5:  60.0000
test acc 28.09

Epoch: [9 | 500] LR: 0.050000
(0/500) Data: 0.005s | Batch: 0.025s | | Loss: 0.2390 | top1:  0.5800 
(1/500) Data: 0.002s | Batch: 0.015s | | Loss: 2.8813 | top1:  34.0000 | top5:  65.0000
privacy res 0.5766666666666667 33.5
(10/500) Data: 0.003s | Batch: 0.018s | | Loss: 0.2318 | top1:  0.6225 
(30/500) Data: 0.003s | Batch: 0.019s | | 

...

privacy res 0 98.5
(0/500) Data: 0.006s | Batch: 0.027s | | Loss: 0.1533 | top1:  0.7850 
(101/500) Data: 0.001s | Batch: 0.013s | | Loss: 0.9896 | top1:  99.0000 | top5:  99.0000
privacy res 0.775 98.5
(10/500) Data: 0.003s | Batch: 0.016s | | Loss: 0.1574 | top1:  0.7800 
(30/500) Data: 0.004s | Batch: 0.017s | | Loss: 0.1454 | top1:  0.8100 
privacy res 0.7966666666666666 97.0
(40/500) Data: 0.003s | Batch: 0.016s | | Loss: 0.1387 | top1:  0.8150 
(60/500) Data: 0.002s | Batch: 0.014s | | Loss: 0.1409 | top1:  0.8100 
privacy res 0.7933333333333333 97.5
(70/500) Data: 0.003s | Batch: 0.015s | | Loss: 0.1543 | top1:  0.7900 
(90/500) Data: 0.003s | Batch: 0.015s | | Loss: 0.1581 | top1:  0.7700 
privacy res 0.7633333333333333 97.5
privacy res 0 98.0
(0/500) Data: 0.005s | Batch: 0.029s | | Loss: 0.1519 | top1:  0.7850 
(201/500) Data: 0.001s | Batch: 0.014s | | Loss: 0.8726 | top1:  98.0000 | top5:  99.0000
privacy res 0.7916666666666666 98.5
(10/500) Data: 0.003s | Batch: 0.017s | |

(90/500) Data: 0.004s | Batch: 0.017s | | Loss: 0.1620 | top1:  0.7650 
privacy res 0.7733333333333333 100.0
privacy res 0 98.0
(0/500) Data: 0.005s | Batch: 0.032s | | Loss: 0.1396 | top1:  0.8200 
(201/500) Data: 0.002s | Batch: 0.014s | | Loss: 0.9591 | top1:  96.0000 | top5:  99.0000
privacy res 0.785 97.0
(10/500) Data: 0.003s | Batch: 0.017s | | Loss: 0.1433 | top1:  0.7875 
(30/500) Data: 0.004s | Batch: 0.018s | | Loss: 0.1490 | top1:  0.8050 
privacy res 0.7883333333333333 98.5
(40/500) Data: 0.005s | Batch: 0.020s | | Loss: 0.1504 | top1:  0.7875 
(60/500) Data: 0.004s | Batch: 0.017s | | Loss: 0.1557 | top1:  0.7700 
privacy res 0.7766666666666666 99.0
(70/500) Data: 0.003s | Batch: 0.015s | | Loss: 0.1710 | top1:  0.7700 
(90/500) Data: 0.003s | Batch: 0.016s | | Loss: 0.1730 | top1:  0.7400 
privacy res 0.7833333333333333 98.5
privacy res 0 97.5
(0/500) Data: 0.004s | Batch: 0.022s | | Loss: 0.1632 | top1:  0.7700 
(301/500) Data: 0.002s | Batch: 0.017s | | Loss: 0.9415 | 

(70/500) Data: 0.003s | Batch: 0.018s | | Loss: 0.1612 | top1:  0.7750 
(90/500) Data: 0.004s | Batch: 0.018s | | Loss: 0.1674 | top1:  0.7500 
privacy res 0.7483333333333333 99.0
privacy res 0 98.5
(0/500) Data: 0.006s | Batch: 0.027s | | Loss: 0.1462 | top1:  0.8050 
(301/500) Data: 0.002s | Batch: 0.016s | | Loss: 0.9105 | top1:  98.0000 | top5:  100.0000
privacy res 0.8033333333333333 98.0
(10/500) Data: 0.004s | Batch: 0.019s | | Loss: 0.1512 | top1:  0.7800 
(30/500) Data: 0.004s | Batch: 0.018s | | Loss: 0.1467 | top1:  0.8050 
privacy res 0.7616666666666667 99.0
(40/500) Data: 0.003s | Batch: 0.020s | | Loss: 0.1486 | top1:  0.7900 
(60/500) Data: 0.004s | Batch: 0.019s | | Loss: 0.1504 | top1:  0.8050 
privacy res 0.7983333333333333 98.0
(70/500) Data: 0.002s | Batch: 0.017s | | Loss: 0.1366 | top1:  0.8150 
(90/500) Data: 0.003s | Batch: 0.018s | | Loss: 0.1651 | top1:  0.7650 
privacy res 0.775 98.0
privacy res 0 98.5
(0/500) Data: 0.005s | Batch: 0.034s | | Loss: 0.1748 | t

(40/500) Data: 0.004s | Batch: 0.020s | | Loss: 0.1446 | top1:  0.7975 
(60/500) Data: 0.003s | Batch: 0.015s | | Loss: 0.1759 | top1:  0.7350 
privacy res 0.7366666666666667 98.5
(70/500) Data: 0.004s | Batch: 0.018s | | Loss: 0.1583 | top1:  0.7750 
(90/500) Data: 0.006s | Batch: 0.022s | | Loss: 0.1375 | top1:  0.8050 
privacy res 0.81 97.0
privacy res 0 100.0
(0/500) Data: 0.005s | Batch: 0.027s | | Loss: 0.1532 | top1:  0.7850 
(401/500) Data: 0.002s | Batch: 0.014s | | Loss: 0.8762 | top1:  100.0000 | top5:  100.0000
privacy res 0.795 97.5
(10/500) Data: 0.004s | Batch: 0.018s | | Loss: 0.1420 | top1:  0.8075 
(30/500) Data: 0.004s | Batch: 0.018s | | Loss: 0.1643 | top1:  0.7350 
privacy res 0.7633333333333333 97.5
(40/500) Data: 0.012s | Batch: 0.026s | | Loss: 0.1414 | top1:  0.8075 
(60/500) Data: 0.003s | Batch: 0.018s | | Loss: 0.1358 | top1:  0.7950 
privacy res 0.7716666666666666 99.0
(70/500) Data: 0.003s | Batch: 0.015s | | Loss: 0.1405 | top1:  0.7925 
(90/500) Data: 0

(10/500) Data: 0.004s | Batch: 0.017s | | Loss: 0.1407 | top1:  0.7950 
(30/500) Data: 0.005s | Batch: 0.020s | | Loss: 0.1437 | top1:  0.8150 
privacy res 0.8083333333333333 99.5
(40/500) Data: 0.004s | Batch: 0.016s | | Loss: 0.1376 | top1:  0.8000 
(60/500) Data: 0.003s | Batch: 0.015s | | Loss: 0.1505 | top1:  0.7900 
privacy res 0.7833333333333333 99.0
(70/500) Data: 0.003s | Batch: 0.018s | | Loss: 0.1436 | top1:  0.8125 
(90/500) Data: 0.003s | Batch: 0.014s | | Loss: 0.1524 | top1:  0.7650 
privacy res 0.7666666666666667 97.5
privacy res 0 99.0
(1/100) Data: 0.189s | Batch: 0.196s | Loss: 4.5000 | top1:  30.0000 | top5:  41.0000
test acc 34.44

Epoch: [104 | 500] LR: 0.000500
(0/500) Data: 0.005s | Batch: 0.028s | | Loss: 0.1237 | top1:  0.8350 
(1/500) Data: 0.002s | Batch: 0.016s | | Loss: 0.9590 | top1:  95.0000 | top5:  98.0000
privacy res 0.8 97.5
(10/500) Data: 0.003s | Batch: 0.017s | | Loss: 0.1541 | top1:  0.7750 
(30/500) Data: 0.003s | Batch: 0.016s | | Loss: 0.1532 